In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
from os import listdir
from os.path import isfile, join
folderpath='C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/'  #folderpath can be changed to a different folder; 

files = [f for f in listdir(folderpath) if isfile(join(folderpath, f))]
files

['event_type.csv',
 'log_feature.csv',
 'resource_type.csv',
 'sample_submission.csv',
 'severity_type.csv',
 'test.csv',
 'train.csv']

##### Manually clean files event_type and resource_type prior to reading in. 

In [131]:
event_type=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/event_type.csv')
log_feature=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/log_feature.csv')
resource_type=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/resource_type.csv')
sample_submission=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/sample_submission.csv')
severity_type=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/severity_type.csv')
test=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/test.csv')
train=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/train.csv')

In [132]:
print(event_type.tail())
print('\n')
print(log_feature.head())
print('\n')
print(resource_type.tail())
print('\n')
print(sample_submission.head())
print('\n')
print(severity_type.head())
print('\n')
print(test.head())
print('\n')
print(train.head())

         id     event_type
34077  3761  event_type 11
34078  8720  event_type 11
34079  6488  event_type 11
34080   878  event_type 11
34081  4464  event_type 11


     id  log_feature  volume
0  6597   feature 68       6
1  8011   feature 68       7
2  2597   feature 68       1
3  5022  feature 172       2
4  5022   feature 56       1


         id    resource_type
22871  3761  resource_type 8
22872  8720  resource_type 8
22873  6488  resource_type 8
22874   878  resource_type 8
22875  4464  resource_type 8


      id  predict_0  predict_1  predict_2
0  11066          0          1          0
1  18000          0          1          0
2  16964          0          1          0
3   4795          0          1          0
4   3392          0          1          0


     id    severity_type
0  6597  severity_type 2
1  8011  severity_type 2
2  2597  severity_type 2
3  5022  severity_type 1
4  6852  severity_type 1


      id      location
0  11066  location 481
1  18000  location 962
2  16964 

In [133]:
print(event_type.info())
print('\n')
print(log_feature.info())
print('\n')
print(resource_type.info())
print('\n')
print(sample_submission.info())
print('\n')
print(severity_type.info())
print('\n')
print(test.info())
print('\n')
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34082 entries, 0 to 34081
Data columns (total 2 columns):
id            34082 non-null int64
event_type    34082 non-null object
dtypes: int64(1), object(1)
memory usage: 532.6+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58671 entries, 0 to 58670
Data columns (total 3 columns):
id             58671 non-null int64
log_feature    58671 non-null object
volume         58671 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.3+ MB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22876 entries, 0 to 22875
Data columns (total 2 columns):
id               22876 non-null int64
resource_type    22876 non-null object
dtypes: int64(1), object(1)
memory usage: 357.5+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11171 entries, 0 to 11170
Data columns (total 4 columns):
id           11171 non-null int64
predict_0    11171 non-null int64
predict_1    11171 non-null int64
predict_2    11171 non-null int64


In [149]:
print('event_type')
print(event_type['id'].nunique())
print(event_type['event_type'].nunique())
print(event_type['event_type'].value_counts())
print('\n')

print('log_feature')
print(log_feature['id'].nunique())
print(log_feature['log_feature'].nunique())
print(log_feature['log_feature'].value_counts())
print('\n')

print('resource_type')
print(resource_type['id'].nunique())
print(resource_type['resource_type'].nunique())
print(resource_type['resource_type'].value_counts())
print('\n')

print('severity_type')
print(severity_type['id'].nunique())
print(severity_type['severity_type'].nunique())
print(severity_type['severity_type'].value_counts())
print('\n')
#All datasets include the total of 18,552 IDs, and log_feature has most categories (386) --- may need further grouping;
#There are 52 categories of event_type, 10 of resource_type, and 5 of severity_type. 

event_type
18552
53
event_type 11    9041
event_type 35    6814
event_type 34    6116
event_type 15    5003
event_type 20    1594
event_type 54     785
event_type 13     682
event_type 42     529
event_type 44     513
event_type 23     482
event_type 14     381
event_type 43     347
event_type 22     254
event_type 50     174
event_type 10     147
event_type 21     141
event_type 32      84
event_type 47      82
event_type 18      82
event_type 30      75
event_type 45      67
event_type 26      66
event_type 24      55
event_type 27      45
event_type 29      45
event_type 40      40
event_type 2       39
event_type 46      38
event_type 8       37
event_type 6       36
event_type 28      32
event_type 5       31
event_type 7       29
event_type 38      21
event_type 3       19
event_type 49      19
event_type 53      19
event_type 39      18
event_type 36      18
event_type 9       18
event_type 19      14
event_type 31      11
event_type 37      10
event_type 12       6
event_type 2

In [190]:
## print(train['fault_severity'].value_counts())
print(train['fault_severity'].nunique())
print('\n')

print(train['id'].nunique())
print('\n')

print(test['id'].nunique())
print('\n')

print(train['location'].value_counts())
print(train['location'].nunique())
print('\n')

print(test['location'].value_counts())
print(test['location'].nunique())
print('\n')
#1039 unique locations in 11,172 test data; 929 unique locations in 7,382 train data. 

3


7381


11171


location 821     85
location 1107    78
location 734     75
location 1008    71
location 126     71
location 810     66
location 600     64
location 704     61
location 242     60
location 124     59
location 1052    58
location 122     56
location 684     56
location 793     55
location 834     54
location 653     53
location 1019    53
location 798     52
location 763     51
location 846     50
location 91      49
location 976     48
location 1100    45
location 471     45
location 962     45
location 102     44
location 1042    43
location 244     41
location 808     40
location 995     40
                 ..
location 589      1
location 987      1
location 592      1
location 582      1
location 250      1
location 1013     1
location 622      1
location 63       1
location 20       1
location 538      1
location 235      1
location 578      1
location 291      1
location 968      1
location 370      1
location 859      1
location 933      1
location 162      1
l

In [257]:
#Convert long-to-wide of each attribute data, and then collapse; 
event_type=event_type.sort_values(by='id', ascending=False) 
event_type_1=pd.get_dummies(event_type, columns=['event_type'], drop_first=True)
event_type_1=event_type_1.groupby('id').sum()
event_type_1['id'] = event_type_1.index
event_type_1.head()

,event_type_event_type 10,event_type_event_type 11,event_type_event_type 12,event_type_event_type 13,event_type_event_type 14,event_type_event_type 15,event_type_event_type 17,event_type_event_type 18,event_type_event_type 19,event_type_event_type 2,...,event_type_event_type 50,event_type_event_type 51,event_type_event_type 52,event_type_event_type 53,event_type_event_type 54,event_type_event_type 6,event_type_event_type 7,event_type_event_type 8,event_type_event_type 9,id
id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [316]:
severity_type=severity_type.sort_values(by='id', ascending=False) 
severity_type_1=pd.get_dummies(severity_type, columns=['severity_type'], drop_first=True)
severity_type_1=severity_type_1.groupby('id').sum()
severity_type_1['id'] = severity_type_1.index
severity_type_1.head()

,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,id
id,,,,,
1,0,0,0,0,1
2,1,0,0,0,2
3,0,0,0,0,3
4,0,0,1,0,4
5,1,0,0,0,5


In [279]:
log_feature=log_feature.sort_values(by='id', ascending=False) 
log_feature_1=pd.pivot_table(log_feature, values='volume', index=['id'], columns=['log_feature'], aggfunc=np.sum, fill_value=0)
log_feature_1['id'] = log_feature_1.index

#double check the conversion by examples (before & after conversion); 
#log_feature.iloc[80:120]
#log_feature_1.iloc[18515:18530, -20:]

In [284]:
resource_type=resource_type.sort_values(by='id', ascending=False) 
resource_type_1=pd.get_dummies(resource_type, columns=['resource_type'], drop_first=True)
resource_type_1=resource_type_1.groupby('id').sum()
resource_type_1['id'] = resource_type_1.index
resource_type_1.head()

,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,resource_type_resource_type 7,resource_type_resource_type 8,resource_type_resource_type 9,id
id,,,,,,,,,,
1,0,0,0,0,0,1,0,1,0,1
2,0,1,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,1,0,3
4,0,1,0,0,0,0,0,0,0,4
5,0,1,0,0,0,0,0,0,0,5


In [317]:
#merge all attribute datasets together (should be 18552 entries each and lastly); 
data=severity_type_1.merge(event_type_1, left_on='id', right_on='id', how='left')
data=data.merge(resource_type_1, left_on='id', right_on='id', how='left')
data=data.merge(log_feature_1, left_on='id', right_on='id', how='left')

In [318]:
data.info() #with all attribute columns, and only 1 id kept; 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18552 entries, 0 to 18551
Columns: 452 entries, severity_type_severity_type 2 to feature 99
dtypes: int64(387), uint8(65)
memory usage: 56.1 MB


In [369]:
#add all attributes to train and test data provided; 
data_train=data.merge(train,  left_on='id', right_on='id', how='right')
data_test=data.merge(test,  left_on='id', right_on='id', how='right')

In [353]:
print(data_train.info())
print(data_test.info())  #test here is for prediction after classifier finalized;  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7381 entries, 0 to 7380
Columns: 454 entries, severity_type_severity_type 2 to fault_severity
dtypes: int64(388), object(1), uint8(65)
memory usage: 22.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11171 entries, 0 to 11170
Columns: 456 entries, severity_type_severity_type 2 to predict_2
dtypes: int64(390), object(1), uint8(65)
memory usage: 34.1+ MB
None


In [392]:
'''
#If tuning paramters, then train/test data defined as belows:
X=data_train.drop(['fault_severity', 'id', 'location'], axis=1)
Y=data_train['fault_severity']
x_train, x_test, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=2345) 

from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(x_train)
xtrain = std_scale.transform(x_train)
xtest  = std_scale.transform(x_test)  
'''

In [398]:
#We will use full train data by different models, and then ensemble from all results, thus we use full train data here; 
xtrain=data_train.drop(['fault_severity', 'id', 'location'], axis=1)
ytrain=data_train['fault_severity']

xtest=data_test.drop(['id', 'location'], axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

rs=300
result=pd.DataFrame()
def classifier(classifier, cname): 
    model=classifier
    global result

    model.fit(xtrain, ytrain)
    pred=model.predict(xtest)
    tpred=model.predict(xtrain)
    
    predproba=pd.DataFrame(model.predict_proba(xtest))
    tpredproba=pd.DataFrame(model.predict_proba(xtrain))
    
    sns.distplot(predproba[1], label=str(cname))  
    plt.title("Distribution of Prediction Probabilities") 
    plt.legend()
    plt.show() ###     
    
    #compare bad predictions of different models; 
    result=pd.concat([result, pd.DataFrame(pred)], axis=1) #to merge the predicted labels back to the dataset;
    result.rename(columns={0:str(cname + '_pred')}, inplace=True)
    
#classifier(LogisticRegression(random_state=rs), 'LogisticRegression')
#classifier(RandomForestClassifier(random_state=rs, n_estimators=150), 'RandomForest')
classifier(XGBClassifier(random_state=rs, learning_rate=0.01, n_estimators=1000, gamma=0.05), 'XGBoost')    

In [408]:
result.groupby(['LogisticRegression_pred', 'RandomForest_pred', 'XGBoost_pred']).size()

LogisticRegression_pred  RandomForest_pred  XGBoost_pred
0                        0                  0               6645
                                            1                205
                                            2                124
                         1                  0               1018
                                            1                711
                                            2                 91
                         2                  0                 86
                                            1                 16
                                            2                335
1                        0                  0                326
                                            1                 70
                                            2                  6
                         1                  0                272
                                            1                376
                                 

In [411]:
def fal_sev(df):
#LogisticRegression_pred  RandomForest_pred  XGBoost_pred;    
    if df['XGBoost_pred']==df['RandomForest_pred']:
        return df['XGBoost_pred']
    elif df['XGBoost_pred']==df['LogisticRegression_pred']:
        return df['XGBoost_pred']
    elif df['RandomForest_pred']==df['LogisticRegression_pred']:
        return df['RandomForest_pred']
    else: 
        return max(df['XGBoost_pred'], df['LogisticRegression_pred'], df['RandomForest_pred'])
    
result['fault_severity']=result.apply(fal_sev, axis=1)

In [415]:
result['fault_severity'].value_counts()/len(result.index)

0    0.755349
1    0.130785
2    0.113866
Name: fault_severity, dtype: float64

In [416]:
train['fault_severity'].value_counts()/len(train.index)

0    0.648151
1    0.253489
2    0.098361
Name: fault_severity, dtype: float64

result_full=pd.concat([data_test['id'], result['fault_severity']], axis=1)
result_full=pd.get_dummies(result_full, columns=['fault_severity'], prefix='predict', drop_first=False)
result_full.tail(20)

In [442]:
result_full.to_csv('C:/Users/Yunsun2/Desktop/Github/Kaggle/Telstra/submissions.csv',index=False)

#### To further improve the results: more feature engineering is definitely needed, e.g. utilize the feature 'location'  which was dropped in this quick analysis, further grouping of categorical features such as log_feature. Upon that, then more paramteter tunning/cross-validation/grid-search can also be applied. 